In [3]:
import spacy
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix)
import seaborn as sns


/home/kayo/Documents/indicador_series/env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
dataframe = pd.read_csv('data/stock_data.csv')

In [21]:
base_treinamento,base_teste = train_test_split(dataframe,test_size=0.7,stratify=dataframe['Sentiment'],random_state=0)

In [22]:
data = {
    'POSITIVO':base_treinamento['Text'].loc[base_treinamento['Sentiment']==1].to_list(),
    'NEGATIVO':base_treinamento['Text'].loc[base_treinamento['Sentiment']==-1].to_list()
}

In [23]:
nlp = spacy.load("en_core_web_sm")

In [24]:
nlp.disable_pipes(['tagger','attribute_ruler','parser'])

['tagger', 'attribute_ruler', 'parser']

In [ ]:
nlp.add_pipe(
    "textcat", 
    config={
        "data": data, 
        "model": "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
        "device": "cpu"
    }
)

In [ ]:
nlp.to_disk("./en_example_pipeline2")

In [ ]:
base_comparar = base_teste.copy()

In [ ]:
base_comparar.reset_index(inplace=True)

In [ ]:
del base_comparar['index']

In [ ]:
for index,dado in base_comparar.iterrows():
  resultado = nlp(dado.Text)._.cats
  if resultado['NEGATIVO'] > resultado['POSITIVO']:
    base_comparar['previsao'].iat[index] = -1
  else:
        base_comparar['previsao'].iat[index] = 1


In [ ]:
matrix = pd.crosstab(base_comparar['Sentiment'],base_comparar['previsao'])

In [ ]:
sns.heatmap(matrix,annot=True);

In [ ]:
accuracy_score(base_comparar['Sentiment'],base_comparar['previsao'])

In [ ]:
nlp('Sensex jumps over, Nifty touches 10,500 as markets recover early losses; Reliance Industries rises nearl')._.cats

In [9]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest, chi2

X, y = load_iris(return_X_y=True)
X.shape

X_new = SelectKBest(chi2, k=2).fit_transform(X, y)

In [15]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [14]:
selecionadas = pd.DataFrame(X_new)
selecionadas['y'] = y
selecionadas

,0,1,y
0,1.4,0.2,0
1,1.4,0.2,0
2,1.3,0.2,0
3,1.5,0.2,0
4,1.4,0.2,0
...,...,...,...
145,5.2,2.3,2
146,5.0,1.9,2
147,5.2,2.0,2
148,5.4,2.3,2
